##own_innference

In [ ]:
# Install required libraries
!pip install tensorflow -q
!pip install opencv-python -q
!pip install scikit-learn -q
!pip install keras -q

# Import the libraries
import tensorflow as tf
import numpy as np
import keras
from PIL import Image
from keras import layers
from scipy.special import erf # Importing erf function for GELU



In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set Image and Model Parameters
patch_height = 8
patch_width = 8
img_height = 128
img_width = 128

# Path for the input image
dataset_path = "/content/drive/MyDrive/drone_vit/drone_dataset/no drone/00000463_(5).jpg"

# Path to the directory where your TFLite weights are stored
weights_base_path = "/content/drive/MyDrive/drone_vit/Outputs_Quantized_Matched_Names/weights/"

# Define paths for all weight files
lp_weight_path = weights_base_path + "lp_weight.txt"
lp_emb_path = weights_base_path + "lp_pos_emb.txt"
mha_wq0_path = weights_base_path + "mha_weight_q0.txt"
mha_wk0_path = weights_base_path + "mha_weight_k0.txt"
mha_wv0_path = weights_base_path + "mha_weight_v0.txt"
mha_wq1_path = weights_base_path + "mha_weight_q1.txt"
mha_wk1_path = weights_base_path + "mha_weight_k1.txt"
mha_wv1_path = weights_base_path + "mha_weight_v1.txt"
mha_wouput_path = weights_base_path + "mha_weight_last.txt"
mlp_d0_w_path = weights_base_path + "mlp_weight0.txt"
mlp_d0_b_path = weights_base_path + "mlp_bias0.txt"
mlp_d1_w_path = weights_base_path + "mlp_weight1.txt"
mlp_d1_b_path = weights_base_path + "mlp_bias1.txt"
fc_d0_w_path = weights_base_path + "fc_weight0.txt"
fc_d0_b_path = weights_base_path + "fc_bias0.txt"
fc_d1_w_path = weights_base_path + "fc_weight1.txt"
fc_d1_b_path = weights_base_path + "fc_bias1.txt"
fc_d2_w_path = weights_base_path + "fc_weight2.txt"
fc_d2_b_path = weights_base_path + "fc_bias2.txt"

print("✅ All paths and parameters are set.")

✅ All paths and parameters are set.


In [ ]:
data_augmentation = keras.Sequential([layers.Resizing(img_width, img_height)])
def gelu_exact(x): return 0.5 * x * (1 + erf(x / np.sqrt(2)))
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)
def patch(img, img_width, img_height, patch_width, patch_height):
    patches = []
    for h in range(0, img_height, patch_height):
        for w in range(0, img_width, patch_width):
            patches.append(img[h:h+patch_height, w:w+patch_width])
    return patches
print("✅ Helper functions defined.")

✅ Helper functions defined.


In [ ]:
def lp(data, patches_num):
    weight = np.loadtxt(lp_weight_path, delimiter=",", dtype=np.float32).reshape(192, 64)
    pos_emb = np.loadtxt(lp_emb_path, delimiter=",", dtype=np.float32).reshape(256, 64)
    embed_img = []
    for i in range(patches_num):
        flattened_patch = data[i].reshape(1, 192)
        weighted_patch = np.dot(flattened_patch, weight)
        embed_patch = weighted_patch + pos_emb[i]
        embed_img.append(embed_patch)
    return embed_img

def ln(data):
    data_matrix = np.array(data).squeeze()
    mean = np.mean(data_matrix, axis=1, keepdims=True)
    stdev = np.std(data_matrix, axis=1, keepdims=True) + 1e-6
    return (data_matrix - mean) / stdev

def mha(data, patches_num):
    wq0, wk0, wv0 = [np.loadtxt(p, delimiter=",", dtype=np.float32).reshape(64, 32) for p in [mha_wq0_path, mha_wk0_path, mha_wv0_path]]
    wq1, wk1, wv1 = [np.loadtxt(p, delimiter=",", dtype=np.float32).reshape(64, 32) for p in [mha_wq1_path, mha_wk1_path, mha_wv1_path]]
    w_out = np.loadtxt(mha_wouput_path, delimiter=",", dtype=np.float32).reshape(64, 64)

    q0, k0, v0 = np.dot(data, wq0), np.dot(data, wk0), np.dot(data, wv0)
    q1, k1, v1 = np.dot(data, wq1), np.dot(data, wk1), np.dot(data, wv1)

    attention0 = softmax(np.dot(q0, k0.T) / np.sqrt(k0.shape[-1]))
    head0 = np.dot(attention0, v0)
    attention1 = softmax(np.dot(q1, k1.T) / np.sqrt(k1.shape[-1]))
    head1 = np.dot(attention1, v1)

    concatenated = np.hstack((head0, head1))
    return np.dot(concatenated, w_out)

def add(input1, input2):
    arr1 = np.array(input1).squeeze()
    arr2 = np.array(input2).squeeze()
    return np.add(arr1, arr2)

def mlp(data):
    w0, b0 = np.loadtxt(mlp_d0_w_path, delimiter=",", dtype=np.float32).reshape(64, 128), np.loadtxt(mlp_d0_b_path, delimiter=",", dtype=np.float32).reshape(1, 128)
    w1, b1 = np.loadtxt(mlp_d1_w_path, delimiter=",", dtype=np.float32).reshape(128, 64), np.loadtxt(mlp_d1_b_path, delimiter=",", dtype=np.float32).reshape(1, 64)
    d0 = gelu_exact(np.dot(data, w0) + b0)
    d1 = gelu_exact(np.dot(d0, w1) + b1)
    return d1

def fc(data):
    flat_data = data.flatten().reshape(1, -1)
    w0, b0 = np.loadtxt(fc_d0_w_path, delimiter=",", dtype=np.float32).reshape(16384, 2048), np.loadtxt(fc_d0_b_path, delimiter=",", dtype=np.float32).reshape(1, 2048)
    w1, b1 = np.loadtxt(fc_d1_w_path, delimiter=",", dtype=np.float32).reshape(2048, 1024), np.loadtxt(fc_d1_b_path, delimiter=",", dtype=np.float32).reshape(1, 1024)
    w2, b2 = np.loadtxt(fc_d2_w_path, delimiter=",", dtype=np.float32).reshape(1024, 2), np.loadtxt(fc_d2_b_path, delimiter=",", dtype=np.float32).reshape(1, 2)
    d0 = gelu_exact(np.dot(flat_data, w0) + b0)
    d1 = gelu_exact(np.dot(d0, w1) + b1)
    d2 = np.dot(d1, w2) + b2
    return d2

print("✅ Core layer simulation functions defined.")

✅ Core layer simulation functions defined.


In [ ]:
def vit_infer_simulation():
    img_data = Image.open(dataset_path)
    augmented = data_augmentation(np.array(img_data))
    aug_img = np.array(augmented).astype("float32") / 255.0
    patches_num = (img_height // patch_height)**2
    patches = patch(aug_img, img_width, img_height, patch_width, patch_height)

    lp_array = lp(patches, patches_num)
    ln1_array = ln(lp_array)
    mha_array = mha(ln1_array, patches_num)
    add1_array = add(lp_array, mha_array)
    ln2_array = ln(add1_array)
    mlp_array = mlp(ln2_array)
    add2_array = add(add1_array, mlp_array)
    ln3_array = ln(add2_array)
    fc_out = fc(ln3_array)
    fc_softmax = softmax(fc_out)

    predicted_class = np.argmax(fc_softmax)

    print("\n--- Simulation Inference (Modified NumPy Script) ---")
    print(f"Image Path: {dataset_path}")
    print(f"Simulated Probabilities: {fc_softmax[0]}")

    print(f"Simulated Predicted Class: {'no drone' if predicted_class == 0 else 'drone'}")

print("✅ Main inference function defined.")

✅ Main inference function defined.


In [ ]:
# Run the complete inference simulation
vit_infer_simulation()


--- Simulation Inference (Modified NumPy Script) ---
Image Path: /content/drive/MyDrive/drone_vit/drone_dataset/no drone/00000463_(5).jpg
Simulated Probabilities: [1. 0.]
Simulated Predicted Class: no drone
